In [ ]:
# Import all the libraries we need for our AI agents project
# Importamos todas las librerías que necesitamos para nuestro proyecto de agentes de IA
from dotenv import load_dotenv  # To load secret keys from .env file / Para cargar claves secretas del archivo .env
from openai import AsyncOpenAI  # To connect to OpenAI API / Para conectar con la API de OpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput  # AI agent tools / Herramientas para agentes de IA
import asyncio  # To run multiple tasks at the same time / Para ejecutar múltiples tareas al mismo tiempo
from typing import Dict  # For type hints / Para indicar tipos de datos
import sendgrid  # To send emails / Para enviar correos electrónicos
import os  # To access system variables / Para acceder a variables del sistema
from sendgrid.helpers.mail import Mail, Email, To, Content  # Email helpers / Ayudantes para correos
from pydantic import BaseModel  # To create data models / Para crear modelos de datos

In [ ]:
# Load environment variables from .env file (override any existing ones)
# Cargamos las variables de entorno desde el archivo .env (sobrescribiendo las existentes)
load_dotenv(override=True)

In [ ]:
# Check if our API keys are properly loaded
# Verificamos si nuestras claves de API están cargadas correctamente

openai_api_key = os.getenv("OPENAI_API_KEY")  # Get OpenAI key / Obtener clave de OpenAI
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")  # Get OpenRouter key / Obtener clave de OpenRouter
openrouter_base_url = os.getenv("OPENROUTER_BASE_URL")  # Get OpenRouter URL / Obtener URL de OpenRouter

# Print first 8 characters of each key to verify they're loaded (security practice)
# Imprimimos los primeros 8 caracteres de cada clave para verificar que están cargadas (práctica de seguridad)
if openai_api_key:
    print(f"openai api key exits an begins with: {openai_api_key[:8]}")
else:
    print("OpenAI api key not set")

if openrouter_api_key:
    print(f"open Router api key exits an begins with: {openrouter_api_key[:8]}")
else:
    print("Open Router api key not set")

if openrouter_base_url:
    print(f"open Router base url exits and is: {openrouter_base_url}")
else:
    print("Open Router api key not set")

In [ ]:
# Create different personalities for our sales agents
# Creamos diferentes personalidades para nuestros agentes de ventas

# Professional and serious sales agent / Agente de ventas profesional y serio
instructions1 = "Eres un agente comercial que trabaja para Avomo Innovations, \
una startup que desarrolla agentes de inteligencia artificial especializados en ciberseguridad. \
Escribes correos fríos profesionales y serios."

# Funny and engaging sales agent / Agente de ventas divertido y cautivador
instructions2 = "Eres un agente comercial con sentido del humor y estilo cautivador que trabaja para Avomo Innovations, \
una startup que desarrolla agentes de inteligencia artificial especializados en ciberseguridad. \
Escribes correos fríos ingeniosos y atractivos que probablemente obtendrán una respuesta."

# Busy and direct sales agent / Agente de ventas ocupado y directo
instructions3 = "Eres un agente comercial muy ocupado que trabaja para Avomo Innovations, \
una startup que desarrolla agentes de inteligencia artificial especializados en ciberseguridad. \
Escribes correos fríos breves y directos al grano."

In [ ]:
# Set up different AI models using OpenRouter (a service that gives access to multiple AI models)
# Configuramos diferentes modelos de IA usando OpenRouter (un servicio que da acceso a múltiples modelos de IA)

# Create OpenAI client using OpenRouter endpoint / Creamos cliente OpenAI usando el endpoint de OpenRouter
openRouter_client = AsyncOpenAI(api_key=openrouter_api_key, base_url=openrouter_base_url)

# Create two different AI models / Creamos dos modelos de IA diferentes
deepseek_model = OpenAIChatCompletionsModel(model="deepseek/deepseek-r1-0528:free", openai_client=openRouter_client)  # DeepSeek model
llama_model = OpenAIChatCompletionsModel(model="meta-llama/llama-3.2-3b-instruct:free", openai_client=openRouter_client)  # Llama model

In [ ]:
# Create three different sales agents with different personalities and AI models
# Creamos tres agentes de ventas diferentes con distintas personalidades y modelos de IA

sales_agent1 = Agent(name="OpenAI sales agent", instructions=instructions1, model="gpt-4o-mini")  # Professional agent using GPT-4
sales_agent2 = Agent(name="Deepseek sales agent", instructions=instructions2, model=deepseek_model)  # Funny agent using DeepSeek
sales_agent3 = Agent(name="Llama sale agent", instructions=instructions3, model=llama_model)  # Direct agent using Llama

message = "Envia un email de venta en frio"  # The task we want all agents to do / La tarea que queremos que hagan todos los agentes

# Run all three agents at the same time and collect their results
# Ejecutamos los tres agentes al mismo tiempo y recolectamos sus resultados
with trace("tutorial - Modelos diferentes en paralelo"):  # Track this operation for debugging
    results = await asyncio.gather(  # Run multiple tasks simultaneously / Ejecutar múltiples tareas simultáneamente
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    # Extract the final output from each agent / Extraemos la salida final de cada agente
    outputs = [result.final_output for result in results]

    # Print each agent's email / Imprimimos el email de cada agente
    for output in outputs:
        print(output + "\n\n")

No olvides chequear: https://platform.openai.com/logs?api=traces

In [ ]:
# Convert our agents into tools that other agents can use
# Convertimos nuestros agentes en herramientas que otros agentes pueden usar

description = "Escribe un email de venta en frío"  # What each tool does / Qué hace cada herramienta

# Create tools from our agents / Creamos herramientas a partir de nuestros agentes
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3]  # Put all tools in a list / Ponemos todas las herramientas en una lista

tools

In [ ]:
# Create a function to send emails using SendGrid service
# Creamos una función para enviar emails usando el servicio SendGrid

@function_tool  # This decorator turns our function into a tool that agents can use / Este decorador convierte nuestra función en una herramienta que los agentes pueden usar
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    # Set up SendGrid client / Configuramos el cliente de SendGrid
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("testingavomo@gamil.com")  # Sender email / Email del remitente
    to_email = To("contactola24podcast@gmail.com")  # Recipient email / Email del destinatario
    content = Content("text/html", html_body)  # Email content in HTML format / Contenido del email en formato HTML
    mail = Mail(from_email, to_email, subject, content).get()  # Create the email / Creamos el email
    sg.client.mail.send.post(request_body=mail)  # Send the email / Enviamos el email
    return {"status": "success"}  # Return success message / Devolvemos mensaje de éxito

In [ ]:
# Create specialized agents for email formatting
# Creamos agentes especializados para formatear emails

# Agent that writes email subjects / Agente que escribe asuntos de email
subject_instructions = "Puedes redactar un asunto para un correo de ventas en frío. \
Se te proporciona un mensaje y debes escribir un asunto para un correo que tenga altas probabilidades de obtener una respuesta."

# Agent that converts text to HTML / Agente que convierte texto a HTML
html_instructions = "Puedes convertir el cuerpo de un correo de texto en un cuerpo de correo en HTML. \
Se te proporciona un cuerpo de correo en texto plano, que podría incluir algo de markdown, \
y debes transformarlo en un cuerpo HTML con un diseño claro, simple y atractivo."

# Create the specialized agents / Creamos los agentes especializados
subject_writer = Agent(name="Email Subject Writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="sbject_writer", tool_description="Escribe un asunto para un email de veta e frío")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="conviertes el cuerpo del mensaje del email en un cuerpo de email en HTML ")

In [ ]:
# Group all email-related tools together
# Agrupamos todas las herramientas relacionadas con email
email_tools = [subject_tool, html_tool, send_html_email]
email_tools

In [ ]:
# Create an agent that manages the entire email sending process
# Creamos un agente que gestiona todo el proceso de envío de emails

instructions = "Eres un formateador y remitente de correos electrónicos. Recibes el cuerpo de un correo que debe ser enviado. \
Primero utilizas la herramienta subject_writer para redactar el asunto del correo, luego usas la herramienta html_converter para convertir el cuerpo a formato HTML. \
Finalmente, usas la herramienta send_html_email para enviar el correo con el asunto y el cuerpo en HTML."

# Create the email manager agent with handoff capability
# Creamos el agente gestor de emails con capacidad de transferencia
emailer_agent = Agent(name="Email manager", 
instructions=instructions, 
handoff_description= "Convierte un email en html y lo envia",  # Description for handoff / Descripción para transferencia
model="gpt-4o-mini")

In [ ]:
# Organize our tools and agents for the main sales manager
# Organizamos nuestras herramientas y agentes para el gerente de ventas principal
tools = [tool1, tool2, tool3]  # Sales agent tools / Herramientas de agentes de ventas
handoffs = [emailer_agent]  # Agents we can hand off to / Agentes a los que podemos transferir trabajo

print(tools)
print(handoffs)

In [ ]:
# Create the main sales manager that orchestrates everything
# Creamos el gerente de ventas principal que orquesta todo

sales_manager_instructions = """
Eres el Responsable de Ventas en Avomo Innovations. Tu objetivo es encontrar el mejor correo de ventas en frío utilizando las herramientas de los agentes de ventas.

Sigue estos pasos cuidadosamente:
1. Genera borradores: Usa las tres herramientas de agentes de ventas para generar tres borradores diferentes de correo. No continúes hasta tener los tres listos.

2. Evalúa y selecciona: Revisa los borradores y elige el único correo que consideres más efectivo. 
Puedes usar las herramientas varias veces si no estás satisfecho con los resultados iniciales.

3. Delegación para envío: Entrega SOLO el correo ganador al agente "Gestor de Correos". Este se encargará del formateo y envío.

Reglas cruciales:
- Debes usar las herramientas de los agentes de ventas para generar los borradores — no los escribas tú mismo.
- Debes entregar exactamente UN solo correo al Gestor de Correos — nunca más de uno.
"""

# Create the main sales manager agent / Creamos el agente gerente de ventas principal
sales_manager = Agent (name="Sales Manager", instructions=sales_manager_instructions, tools=tools, handoffs=handoffs, model="gpt-4o-mini")

# Run the sales manager with tracing for debugging
# Ejecutamos el gerente de ventas con trazado para debugging
with trace("Sales Manager"):
    result = await Runner.run(sales_manager, "Envia un email de venta en frio")
    print(result.final_output)

In [ ]:
# Create a security system to prevent agents from using personal names inappropriately
# Creamos un sistema de seguridad para evitar que los agentes usen nombres personales inapropiadamente

# Define what information our security check should return
# Definimos qué información debe devolver nuestra verificación de seguridad
class NameCheckOutput (BaseModel):
    is_name_in_message : bool  # Whether a name was found / Si se encontró un nombre
    name: str  # The name that was found / El nombre que se encontró

# Create an agent that checks for personal names in messages
# Creamos un agente que verifica nombres personales en los mensajes
guardrail_agent = Agent (
    name = "Name check",
    instructions="Comprueba si el usuario está incluyendo el nombre personal de alguien en lo que quiere que hagas.",
    output_type=NameCheckOutput,  # Force structured output / Forzamos salida estructurada
    model = "gpt-4o-mini"
)

In [ ]:
# Create a security guard function that blocks inappropriate requests
# Creamos una función de guardia de seguridad que bloquea solicitudes inapropiadas

@input_guardrail  # This decorator makes this function act as a security check / Este decorador hace que esta función actúe como verificación de seguridad
async def guardrail_against_name(ctx, agent, message):
    # Run our name-checking agent / Ejecutamos nuestro agente verificador de nombres
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    # If a name is found, trigger the security block / Si se encuentra un nombre, activamos el bloqueo de seguridad
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
# Create a protected version of our sales manager with security checks
# Creamos una versión protegida de nuestro gerente de ventas con verificaciones de seguridad

careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]  # Add our security check / Añadimos nuestra verificación de seguridad
)

# Try to run with a message that contains a personal name (this should be blocked)
# Intentamos ejecutar con un mensaje que contiene un nombre personal (esto debería bloquearse)
message = "Escribe un email de venta en frio para 'Querido CEO' enviado por Expe Avomo"

with trace("Best Sales Email"):
    result = await Runner.run(careful_sales_manager, message)